<a href="https://colab.research.google.com/github/wvls123/colab/blob/main/%D0%9D%D0%B5%D0%B9%D1%80%D0%BE_%D1%80%D0%B5%D0%BF%D0%B5%D1%82%D0%B8%D1%82%D0%BE%D1%80_v4_%D0%92%D0%B5%D0%B1%D0%B8%D0%BD%D0%B0%D1%80_14_%D0%BC%D0%B0%D1%8F_%D0%9E%D1%81%D0%BD%D0%BE%D0%B2%D1%8B_Python_%D0%BE%D1%82_GPT_4_%D0%94%D0%B5%D0%BD%D1%8C_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Занятие Python на ChatGPT. День 1**

# Настройка Нейро-репетитора

## Параметры

In [ ]:
# Примеры использования функции
tema = 'Основы Python'
colab_link = "https://colab.research.google.com/drive/162DEeBL1hQjnAdOiyt8unN1bT8AQ0W6a?usp=sharing"
notebook_filename = 'notebook_colab.ipynb'  # укажите путь к вашему ноутбуку
output_directory = '/content/colab_notebooks/media'  # укажите директорию для сохранения изображений
usd_price = 86

# Очистка истории
# history = []
HISTORY_FILE = 'history.json'  # Имя файла для хранения истории
history_cost = []

## Установка библиотек

In [ ]:
%autosave 3

Autosaving every 3 seconds


In [ ]:
!pip install -q nbformat==5.10.4
!pip install -q gdown
!pip install -q langchain==0.2.6 tiktoken==0.7.0 faiss-cpu==1.8.0 langchain-community==0.2.6 nltk==3.8.1 openai==1.35.10

In [ ]:
!pip install -q langchainhub==0.1.20 langchain-openai==0.1.14 #langchain-chroma==0.1.2

In [ ]:
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

## Системные функции 1

In [ ]:
import IPython
from IPython.display import HTML, Javascript
import json
import os
import requests

HISTORY_FILE = 'history.json'  # Имя файла для хранения истории

def escape_js_string(s):
    return s.replace("\\", "\\\\").replace("'", "\\'").replace('"', '\\"').replace("\n", "\\n").replace("\r", "\\r").replace("`", "\\`").replace("$", "\\$")

def save_history(history):
    with open(HISTORY_FILE, 'w') as f:
        json.dump(history, f)

def load_history():
    if os.path.exists(HISTORY_FILE):
        with open(HISTORY_FILE, 'r') as f:
            history = json.load(f)
        return history
    else:
        return []

# Очистка истории
history = []
save_history(history)
# # Загружаем историю при запуске
# history = load_history()

# Generate HTML for displaying previous conversation history
def generate_history_html(history, context=True):
    global tutor_chat_id
    history_html = ""

    for entry in history:
        user_message = escape_js_string(entry['user'])
        if user_message:
          history_html += f"""
          <div class='message user'><pre style="white-space: pre-wrap; word-wrap: break-word;">{user_message}</pre></div>
          """
        bot_message = json.loads(entry['bot']) if entry['bot'] else None  # Обработка случая, когда нет ответа бота
        if bot_message:
          history_html += f"""
          <div class='message bot'><pre style="white-space: pre-wrap; word-wrap: break-word;">Нейро-репетитор: {bot_message}</pre></div>
          """
    if context:
      # Получение содержимого ноутбука
      context_notebook = get_notebook_content(colab_link, notebook_filename, output_directory)
      start_id = tutor_chat_id-1
      end_id = tutor_chat_id
      fragment = extract_fragment(context_notebook, start_id, end_id)
      initial_question = get_question(fragment)  # Получаем первое сообщение бота
      # print('fragment ', fragment)
      history.append({'user': '', 'bot': json.dumps(initial_question)})  # Сохраняем в виде JSON
      save_history(history)  # Сохраняем историю в файл
      initial_question = escape_js_string(initial_question)
      history_html += f"""
      <div class='message bot'><pre style="white-space: pre-wrap; word-wrap: break-word;">Нейро-репетитор: {initial_question}</pre></div>
      """
    return history_html

def update_chat_interface(history, context):
    history_html = generate_history_html(history, context)
    js = """
    function updateChat(history_html) {
        var chatbox = document.getElementById('chatbox');
        chatbox.innerHTML = history_html;
        chatbox.scrollTop = chatbox.scrollHeight;  // Автопрокрутка вниз
    }
    """
    # js += f"updateChat(`{history_html}`);"
    js += "updateChat(`{}`);".format(escape_js_string(history_html))
    display(Javascript(js))


def call_python(data):
    global history, tema, system_answer, user_answer, history_cost
    try:
        if isinstance(data, dict) and data.get("sender") == "user":
            # Обработка и сохранение сообщения пользователя
            user_msg = data["msg"]
            history.append({'user': user_msg, 'bot': None})
            save_history(history)
            # Перезагрузка истории чата
            update_chat_interface(history, context=False)
            return

        # Placeholder for answer animation
        js_placeholder = """
        var chatbox = document.getElementById('chatbox');
        var botDiv = document.createElement('div');
        botDiv.className = 'message bot';
        botDiv.id = 'temp_response';
        botDiv.innerHTML = '<pre style="white-space: pre-wrap; word-wrap: break-word;">Нейро-репетитор: Печатает...</pre>';
        chatbox.appendChild(botDiv);
        chatbox.scrollTop = chatbox.scrollHeight;
        """
        display(Javascript(js_placeholder))

        result_get_answer, check_done = get_answer(data["msg"], history, tema, system_answer, user_answer)

        try:
          answer = json.loads(result_get_answer.choices[0].message.content)
        except Exception:
          answer = json.loads(result_get_answer)

        formatted_answer = answer.get("answer", 'Не удалось получить ответ') if "error" not in answer else answer["error"]
        if not check_done:
          initial_question_end = get_question_end(formatted_answer)  # Получаем последнее сообщение бота
          formatted_answer = formatted_answer + '\n\n' + initial_question_end  # Добавляем последнее сообщение бота в ответ
          # print("Генерация ответа на вопрос")
          cost_details = calculate_cost(result_get_answer, usd_price)
          history_cost.append({'answer': cost_details})

        history[-1]['bot'] = json.dumps(formatted_answer)  # Обновляем последнее сообщение пользователя с ответом бота
        save_history(history)  # Сохраняем историю в файл

        # Перезагрузка истории чата
        update_chat_interface(history, context=False)
    except Exception as e:
        js = f"""
        var chatbox = document.getElementById('chatbox');
        var botDiv = document.createElement('div');
        botDiv.className = 'message bot';
        botDiv.innerHTML = '<pre style="white-space: pre-wrap; word-wrap: break-word;">Произошла ошибка: {escape_js_string(str(e))}</pre>';
        chatbox.appendChild(botDiv);
        chatbox.scrollTop = chatbox.scrollHeight;
        console.log('Error in call_python:', '{escape_js_string(str(e))}');
        """
        display(Javascript(js))

# Регистрируем функцию для вызова из JavaScript
def call_python_from_js(data):
    call_python(data)

from google.colab import output
output.register_callback('call_python', call_python_from_js)

def display_chat_interface(history):
    # update_chat_interface(initial_history, context=True)
    history_html = generate_history_html(history, context=True)

    display(HTML("""
    <style>
    #chatbox {
        width: 600px;
        height: 600px;
        border: 1px solid #ccc;
        overflow-y: scroll;
        padding: 5px;
        background-color: #f0f0f0;
    }
    .message {
        padding: 5px;
        margin-bottom: 5px;
        border-radius: 10px;
    }
    .user {
        background-color: #dcf8c6;
        margin-left: 20%;
        text-align: right;
    }
    .bot {
        background-color: #ffffff;
        margin-right: 20%;
        text-align: left;
        white-space: pre-wrap;
        word-wrap: break-word;
    }
    input, button {
        padding: 10px;
        margin: 5px;
        border-radius: 5px;
        border: 1px solid #ccc;
    }
    button {
        cursor: pointer;
        background-color: #4CAF50;
        color: white;
    }
    #user_input {
        width: 470px;
    }
    #send_button {
        width: 100px;
    }
    </style>
    <div id="chatbox">""" + history_html + """</div>
    <input type="text" id="user_input" placeholder="Введите сообщение...">
    <button id="send_button" onclick="sendMessage()">Отправить</button>

    <script>
    function sendMessage() {
        var input = document.getElementById('user_input');
        var chatbox = document.getElementById('chatbox');
        var userMessage = input.value;
        input.value = "";  // Очищаем поле ввода
        if (userMessage.trim() === "") return;  // Ничего не делаем, если сообщение пустое

        // Отображаем сообщение пользователя
        var userDiv = document.createElement('div');
        userDiv.className = 'message user';
        userDiv.innerHTML = '<pre style="white-space: pre-wrap; word-wrap: break-word;">' + userMessage + '</pre>';
        chatbox.appendChild(userDiv);

        chatbox.scrollTop = chatbox.scrollHeight;  // Автопрокрутка вниз

        // Сохраняем сообщение пользователя в истории в Python
        google.colab.kernel.invokeFunction('call_python', [{"msg": userMessage, "sender": "user"}], {});

        // Запрашиваем ответ от бота у Python
        google.colab.kernel.invokeFunction('call_python', [{"msg": userMessage, "sender": "bot"}], {});
        console.log('Message sent to Python:', userMessage);
    }

    // Добавляем обработчик события для клавиши Enter
    document.getElementById('user_input').addEventListener('keydown', function(event) {
        if (event.key === 'Enter') {
            sendMessage();
        }
    });

    // Функция для прокрутки вниз
    function scrollToBottom() {
        var chatbox = document.getElementById('chatbox');
        chatbox.scrollTop = chatbox.scrollHeight;
    }

    // Прокрутка вниз после загрузки истории
    window.onload = function() {
        scrollToBottom();
    };
    </script>
    """))

## Системные функции 2

In [ ]:
import re
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

def authenticate_and_create_drive_instance():
    # Аутентификация в Google Colab
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    return GoogleDrive(gauth)

def extract_file_id(link):
    colab_pattern = re.compile(r'(?:drive|d)/([a-zA-Z0-9-_]+)')
    drive_pattern = re.compile(r'/d/([a-zA-Z0-9-_]+)')

    colab_match = colab_pattern.search(link)
    drive_match = drive_pattern.search(link)

    if colab_match:
        return colab_match.group(1)
    elif drive_match:
        return drive_match.group(1)
    else:
        raise ValueError("Неподдерживаемая ссылка. Пожалуйста, предоставьте ссылку на Google Colab или Google Drive.")

def download_ipynb_from_link(link, output_filename):
    file_id = extract_file_id(link)
    drive = authenticate_and_create_drive_instance()

    # Получение файла по ID и сохранение его
    file = drive.CreateFile({'id': file_id})
    file.GetContentFile(output_filename)
    print(f"Файл сохранен как {output_filename}")

# # Примеры использования функции
# colab_link = "https://colab.research.google.com/drive/1CWsNmxUO3LGgF_r2Tyv8WOM_5OzYAkt6?usp=sharing"
# drive_link = "https://drive.google.com/file/d/1jL49juy_Pl9OWGjOmwwFoV5boXG6thQN/view?usp=drive_link"

# # Скачивание файлов
# download_ipynb_from_link(colab_link, "notebook_colab.ipynb")
# download_ipynb_from_link(drive_link, "notebook_drive.ipynb")

In [ ]:
from google.colab import _message
import nbformat
import re
from datetime import datetime
import os
import base64
from io import BytesIO
from PIL import Image
import time

# Функция сохранения медиаданных из ячеек ноутбука и замены на пути к файлам
def save_media(media_data, output_dir, media_count, media_type, file_name):
    media_bytes = base64.b64decode(media_data)
    ext = {
        'image/png': 'png',
        'image/jpeg': 'jpg',
        'image/gif': 'gif',
        'audio/wav': 'wav',
        'audio/mpeg': 'mp3',
        'video/mp4': 'mp4'
    }[media_type]
    media_filename = f"{output_dir}/{file_name}_{media_type.split('/')[0]}_{media_count}.{ext}"
    with open(media_filename, 'wb') as f:
        f.write(media_bytes)
    return media_filename

In [ ]:
# Функция для чтения информации о ячейках ноутбука из сессии колаба
def read_notebook_from_data():
    # Получение .ipynb данных
    response = _message.blocking_request('get_ipynb', request='', timeout_sec=60)
    notebook = nbformat.from_dict(response['ipynb'])

    notebook_info = []

    # Проход по всем ячейкам
    for cell in notebook.cells:
        cell_info = {
            'cell_type': cell.cell_type,
            'source': cell.source,
            'outputs': cell.get('outputs', []),
            'execution_count': cell.get('execution_count', None),
            'metadata': cell.get('metadata', {})
        }

        if cell.cell_type == 'code':
            # Добавление информации о запуске ячейки
            cell_info['executed'] = cell_info['execution_count'] is not None

            execution_info = cell_info['metadata'].get('executionInfo', {})
            cell_info['executed_by'] = execution_info.get('user', {}).get('displayName', 'Unknown')

            timestamp = execution_info.get('timestamp', None)
            if timestamp:
                cell_info['executed_at'] = datetime.fromtimestamp(timestamp / 1000).strftime('%Y-%m-%d %H:%M:%S')
            else:
                cell_info['executed_at'] = 'Unknown'

        notebook_info.append(cell_info)

    return notebook_info

def process_notebook_from_data(notebook_filename, output_directory):
    file_name_without_extension = os.path.splitext(os.path.basename(notebook_filename))[0]
    os.makedirs(output_directory, exist_ok=True)
    notebook_data = read_notebook_from_data()
    output_str = ""
    media_count = 0

    media_types = ['image/png', 'image/jpeg', 'image/gif', 'audio/wav', 'audio/mpeg', 'video/mp4']

    for cell_info in notebook_data:
        output_str += f"Cell type: {cell_info['cell_type']}\n"
        if cell_info['cell_type'] == 'markdown' or cell_info['cell_type'] == 'code':
            source = cell_info['source']
            # Проверка типа данных source и преобразование в строку при необходимости
            if isinstance(source, list):
                source = ''.join(source)
            elif not isinstance(source, str):
                source = str(source)

            for media_type in media_types:
                pattern = re.compile(rf'data:{media_type};base64,([a-zA-Z0-9+/=]+)')
                matches = pattern.findall(source)
                for match in matches:
                    media_data = match
                    media_path = save_media(media_data, output_directory, media_count, media_type, file_name_without_extension)
                    source = source.replace(f"data:{media_type};base64,{media_data}", media_path)
                    media_count += 1
            output_str += f"Source: {source}\n"
        if cell_info['cell_type'] == 'code':
            output_str += f"Execution count: {cell_info['execution_count']}\n"
            output_str += f"Executed: {cell_info['executed']}\n"
            output_str += f"Executed by: {cell_info['executed_by']}\n"
            output_str += f"Executed at: {cell_info['executed_at']}\n"
            for output in cell_info['outputs']:
                if output.output_type in ('display_data', 'execute_result'):
                    for media_type in media_types:
                        if media_type in output.data:
                            media_data = output.data[media_type]
                            media_path = save_media(media_data, output_directory, media_count, media_type, file_name_without_extension)
                            output_str += f"Output {media_type.split('/')[0]} saved at: {media_path}\n"
                            output.data[media_type] = media_path
                            media_count += 1
                elif output.output_type == 'stream' and 'text' in output:
                    output_text = output['text']
                    if len(output_text) > 2000:
                        output_text = output_text[-2000:]
                    output_str += f"Outputs: {output_text}\n"
                else:
                    output_str += f"Outputs: {output}\n"
        output_str += '-' * 30 + '\n'
    return output_str

In [ ]:
import nbformat
import base64
from io import BytesIO
from PIL import Image
import os
from datetime import datetime
import time
import re

# Считываем содержимое файла ноутбука
def read_notebook_text_format(notebook_filename):
  with open(notebook_filename, 'r') as file:
      # Читаем содержимое файла и сохраняем его в переменную
      resheniye = file.read()
  return resheniye


# Функция для чтения информации о ячейках ноутбука
def read_notebook(filename):
    # Открытие .ipynb файла
    with open(filename, 'r', encoding='utf-8') as f:
        notebook = nbformat.read(f, as_version=4)

    notebook_info = []

    # Проход по всем ячейкам
    for cell in notebook.cells:
        cell_info = {
            'cell_type': cell.cell_type,
            'source': cell.source,
            'outputs': cell.get('outputs', []),
            'execution_count': cell.get('execution_count', None),
            'metadata': cell.get('metadata', {})
        }

        if cell.cell_type == 'code':
            # Добавление информации о запуске ячейки
            cell_info['executed'] = cell_info['execution_count'] is not None

            execution_info = cell_info['metadata'].get('executionInfo', {})
            cell_info['executed_by'] = execution_info.get('user', {}).get('displayName', 'Unknown')

            timestamp = execution_info.get('timestamp', None)
            if timestamp:
                cell_info['executed_at'] = datetime.fromtimestamp(timestamp / 1000).strftime('%Y-%m-%d %H:%M:%S')
            else:
                cell_info['executed_at'] = 'Unknown'

        notebook_info.append(cell_info)

    return notebook_info

def process_notebook(notebook_filename, output_directory):
    file_name_without_extension = os.path.splitext(os.path.basename(notebook_filename))[0]
    os.makedirs(output_directory, exist_ok=True)
    notebook_data = read_notebook(notebook_filename)
    output_str = ""
    media_count = 0

    media_types = ['image/png', 'image/jpeg', 'image/gif', 'audio/wav', 'audio/mpeg', 'video/mp4']

    for cell_info in notebook_data:
        output_str += f"Cell type: {cell_info['cell_type']}\n"
        if cell_info['cell_type'] == 'markdown' or cell_info['cell_type'] == 'code':
            source = cell_info['source']
            for media_type in media_types:
                pattern = re.compile(rf'data:{media_type};base64,([a-zA-Z0-9+/=]+)')
                matches = pattern.findall(source)
                for match in matches:
                    media_data = match
                    media_path = save_media(media_data, output_directory, media_count, media_type, file_name_without_extension)
                    source = source.replace(f"data:{media_type};base64,{media_data}", media_path)
                    media_count += 1
            output_str += f"Source: {source}\n"
        if cell_info['cell_type'] == 'code':
            output_str += f"Execution count: {cell_info['execution_count']}\n"
            output_str += f"Executed: {cell_info['executed']}\n"
            output_str += f"Executed by: {cell_info['executed_by']}\n"
            output_str += f"Executed at: {cell_info['executed_at']}\n"
            for output in cell_info['outputs']:
                if output.output_type in ('display_data', 'execute_result'):
                    for media_type in media_types:
                        if media_type in output.data:
                            media_data = output.data[media_type]
                            media_path = save_media(media_data, output_directory, media_count, media_type, file_name_without_extension)
                            output_str += f"Output {media_type.split('/')[0]} saved at: {media_path}\n"
                            output.data[media_type] = media_path
                            media_count += 1
                elif output.output_type == 'stream' and 'text' in output:
                    output_text = output['text']
                    if len(output_text) > 2000:
                        output_text = output_text[-2000:]
                    output_str += f"Outputs: {output_text}\n"
                else:
                    output_str += f"Outputs: {output}\n"
        output_str += '-' * 30 + '\n'
    return output_str

# # # Пример использования функции
# # # Чтение и фильтрация информации из файла ноутбука
# # notebook_filename = '/content/notebook_2.ipynb'  # укажите путь к вашему ноутбуку
# # output_directory = '/content/colab_notebooks/media2'  # укажите директорию для сохранения изображений


# # # Получение и вывод информации о ноутбуке
# # output_str = process_notebook(notebook_filename, output_directory)
# # print(output_str)

## Системные функции 3

In [ ]:
# # Скачивание файла ноутбука по ссылке
# def get_notebook_content(colab_link, notebook_filename, output_directory):
#   download_ipynb_from_link(colab_link, notebook_filename)

#   # Чтение и фильтрация информации из файла ноутбука
#   output_str = process_notebook(notebook_filename, output_directory)
#   return output_str

In [ ]:
# Получение данных ноутбука из сессии
def get_notebook_content(colab_link, notebook_filename, output_directory):
  # Чтение и фильтрация информации из файла ноутбука
  output_str = process_notebook_from_data(notebook_filename, output_directory)
  return output_str

In [ ]:
def calculate_cost(complition_qustions, usd_price, verbose=False):
    """
    Функция для расчета стоимости входных и выходных токенов, а также общей стоимости запроса.

    :param complition_qustions: Объект с информацией о токенах (предполагается, что он имеет атрибуты usage.prompt_tokens и usage.completion_tokens)
    :param usd_price: Цена одного доллара в рублях
    :return: Словарь с рассчитанной стоимостью
    """
    prompt_tokens_cost = (int(complition_qustions.usage.prompt_tokens) * 5) / 1000000 * usd_price
    completion_tokens_cost = (int(complition_qustions.usage.completion_tokens) * 15) / 1000000 * usd_price
    total_cost = prompt_tokens_cost + completion_tokens_cost

    cost_details = {
        "prompt_tokens_cost": prompt_tokens_cost,
        "completion_tokens_cost": completion_tokens_cost,
        "total_cost": total_cost
    }
    if verbose:
      print(f"Цена {complition_qustions.usage.prompt_tokens} входных токенов, руб: {prompt_tokens_cost}")
      print(f"\nЦена {complition_qustions.usage.completion_tokens} выходных токенов, руб: {completion_tokens_cost}")
      print(f"\nЦена запроса, руб: {total_cost}")

    return cost_details


def calculate_total_cost(data):
    # Инициализируем переменную для общей стоимости
    total_cost = 0

    # Проходим по каждому элементу списка и извлекаем значение 'total_cost'
    for item in data:
        for key, value in item.items():
            total_cost += value['total_cost']

    # Печатаем общую стоимость
    print(f"Общая стоимость всех вызовов: {total_cost:.5f} руб")

In [ ]:
import re

def extract_fragment(context, start_id, end_id):
    start_phrase = f'tutor_chat_id = {start_id}'
    end_phrase = f'tutor_chat_id = {end_id}'

    # Использование регулярного выражения для поиска фрагмента текста между start_phrase и end_phrase
    pattern = re.compile(re.escape(start_phrase) + r'(.*?)' + re.escape(end_phrase), re.DOTALL)
    match = pattern.search(context)

    # Если совпадение найдено, вернем фрагмент
    if match:
        return match.group(1).strip()
    else:
        return None

# # Пример использования функции
# start_id = 2
# end_id = 3
# fragment = extract_fragment(context_notebook, start_id, end_id)
# print(fragment)

In [ ]:
import openai
from datetime import datetime

import json
import numpy as np

def convert_history_to_readable_text(history):
    readable_history = []
    for entry in history:
        user_message = entry.get('user', '')
        bot_message = entry.get('bot', '')
        try:
            # Декодируем обратно из JSON-формата и удаляем кавычки
            decoded_bot_message = json.loads(bot_message)
        except Exception as e:
            # print(f"Ошибка декодирования JSON: {e}")
            decoded_bot_message = ''
        readable_history.append(f"User: {user_message}\nTutor: {decoded_bot_message}\n")
    return "\n".join(readable_history)


def get_question(context_notebook=None):
  global history_cost

  complition_questions = generate_question(history, tema, system_question, user_initial_question, context_notebook)
  questions = json.loads(complition_questions.choices[0].message.content)['questions']
  idx = np.random.randint(0, len(questions))
  # print("Ответ: ", json.loads(complition_questions.choices[0].message.content)['question'])
  # print()
  # print("Генерация первого вопроса")
  cost_details = calculate_cost(complition_questions, usd_price)
  history_cost.append({'initial_question_first': cost_details})

  return questions[idx]


def get_question_end(last_answer, context_notebook=None):
  global history_cost

  temp_history = history.copy()
  temp_history[-1]['bot'] = json.dumps(last_answer)

  complition_questions = generate_question(temp_history, tema, system_question, user_question, context_notebook)
  questions = json.loads(complition_questions.choices[0].message.content)['questions']
  idx = np.random.randint(0, len(questions))

  # print("Генерация вопроса в конце ответа")
  cost_details = calculate_cost(complition_questions, usd_price)
  history_cost.append({'initial_question_end': cost_details})

  return questions[idx]


def generate_question(history, tema, system_prompt, user_instruction, context_notebook=None):

    history_readable = convert_history_to_readable_text(history)

    if context_notebook is None:
      messages = [
          {
              "role": "system",
              "content": system_prompt.format(
                  tema=tema,
                  datetime_now=datetime.now(),
              )
          },
          {
              "role": "user",
              "content": user_instruction.format(
                  history=history_readable
              )
          }
      ]
    else:
      messages = [
          {
              "role": "system",
              "content": system_prompt.format(
                  tema=tema,
                  datetime_now=datetime.now(),
              )
          },
          {
              "role": "user",
              "content": user_instruction.format(
                  history=history_readable,
                  context=context_notebook
              )
          }
      ]

    client = openai.OpenAI()
    completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    temperature=1,
    max_tokens=300,
    seed=43,
    response_format={"type": "json_object"},
)

    return completion


def get_answer(question, history, tema, system_prompt, user_instruction):
    # global colab_link, notebook_filename, output_directory

    chek_done = json.loads(generate_question(history, tema, system_decision, user_decision).choices[0].message.content)
    # print("Завершение темы:")
    # print(chek_done['decision'], type(chek_done['decision']))
    history_readable = convert_history_to_readable_text(history)

    if chek_done['decision']:
      return json.dumps({'answer':  chek_done['comment']}), chek_done['decision']

    else:
      # Получение содержимого ноутбука
      context_notebook = get_notebook_content(colab_link, notebook_filename, output_directory)
      messages = [
          {
              "role": "system",
              "content": system_prompt.format(
                  tema=tema,
                  datetime_now=datetime.now(),
              )
          },
          {
              "role": "user",
              "content": user_instruction.format(
                  context=context_notebook,
                  history=history_readable,
                  question=question
              )
          }
      ]

      client = openai.OpenAI()
      completion = client.chat.completions.create(
      model="gpt-4o",
      messages=messages,
      temperature=0.1,
      max_tokens=3300,
      seed=43,
      response_format={"type": "json_object"},
    )

    return completion, chek_done['decision']

## Промпты

In [ ]:
system_question = """Вы вежливый, умный и знающий репетитор, специализирующийся на программировании, особенно на искусственном интеллекте. Вы работаете в среде ноутбука google colaboratory.
Ваша основная роль — заботливо спросить ученика так, чтобы оценить, понял ли он изложенный материал.
Ваши вопросы должны быть ясными, интересными, с целью выявить понимание материала.
Текущая дата и время: {datetime_now}
Текущая тема изучаемого ноутбука google colab: {tema}
"""

user_initial_question = """На основе контекста раздела ноутбука google colab и истории переписки напиши вопросы, которые помогут сформировать обратную связь от ученика по изучаемому разделу.
Вопрос должен ставится коротко чтобы ответ на вопрос в целом сводился к понятно/не понятно. Напишите 5 вариантов вопросов. Ответ предоставьте в формате Json с ключом questions со значением list() и значениями string.

Содержимое пройденного раздела ноутбука google colab:

{context}

История переписки с учеником:

{history}"""

user_question = """На основе истории переписки (последним заданным вопросам и ответам) напиши вопросы, которые помогут сформировать обратную связь от ученика по изучаемому разделу.
Вопрос должен ставится коротко чтобы ответ на вопрос в целом сводился к понятно/не понятно. Напишите 5 вариантов вопросов. Ответ предоставьте в формате Json с ключом questions со значением list() и значениями string.

История переписки с учеником:

{history}"""

system_decision = """Вы вежливый, умный и знающий репетитор, специализирующийся на программировании, особенно на искусственном интеллекте.
Ваша задача — четко определить, все ли понятно ученику, или нужно остановиться и изучить вопрос более подробно.
Текущая дата и время: {datetime_now}
Текущее тема изучаемого раздела блокнота: {tema}
"""

user_decision = """На основе предоставленной истории переписки (последним заданным вопросам и ответам) примите решение о продолжении обучения или остановки на данном этапе для более подробного изучения.
Ответ должен быть в формате Json с ключом decision и значением True (если ответ пользователя утвердитетельный и не требующий ответа) или False (если пользователю нужно дать дополнительный ответ) и ключом comment со значением текстового комментария (вежливым и мотивирующим высказыванием по теме).

История переписки с учеником:

{history}
"""


system_answer = """Вы выдающийся преподаватель и ответственный за предоставление полных и понятных ответов на вопросы ученика по программированию, искусственному интеллекту, нейронным сетям и смежным областям.
Ваши ответы должны основываться на содержимом изучаемого ноутбука google colab (код и вывод ячеек), теме занятия и истории сообщений с учеником. Раздел и код '# Настройка Нейро-репетитора' а также строка display_chat_interface(history) из контекста к теме занятия не относится и является системной.
Текущая дата и время: {datetime_now}
Текущая тема изучаемого ноутбука google colab: {tema}
"""

user_answer = """На основе предоставленной истории переписки и контекста ноутбука google colab, ответьте на вопрос ученика.
Ответ должен быть детальным и понятным рамках изучаемого материала.
Формат ответа: Json с ключом answer и значением string.

Вопрос ученика:

{question}

История переписки с учеником:

{history}

Содержимое ноутбука google colab:

{context}
"""

tutor_chat_id = 0

# **1. Введение в Python**

## **Краткая история и преимущества языка**

### История

Python был создан Гвидо ван Россумом в конце 1980-х годов и впервые выпущен в 1991 году. Название языка было вдохновлено британским комедийным шоу "Monty Python's Flying Circus". Python изначально разрабатывался как язык с простым и интуитивно понятным синтаксисом, что сделало его легким в освоении для начинающих и мощным инструментом для профессионалов.

### Преимущества Python

1. **Простота и читаемость кода**:
   - Python ориентирован на чистый, понятный и легко читаемый код.
   - Это делает его идеальным языком для тех, кто только начинает изучать программирование.
   - Благодаря простому синтаксису код становится интуитивно понятным даже без глубоких знаний в программировании.

2. **Богатая стандартная библиотека**:
   - Стандартная библиотека Python включает большое количество готовых инструментов, которые называются модулями и пакетами.
   - Они позволяют решать множество задач без необходимости писать сложный код с нуля.
   - К примеру, библиотека `math` предоставляет математические функции, a  `matplotlib` предоставляет инструменты для построения графиков и визуализации данных.

3. **Сообщество и экосистема**:
   - Python имеет активное и дружелюбное сообщество разработчиков по всему миру.
   - Существует огромная экосистема библиотек и инструментов для веб-разработки, анализа данных, машинного обучения, автоматизации и многих других сфер.
   - Многие библиотеки создаются и поддерживаются этим сообществом, что обеспечивает их актуальность и качество.

4. **Кросс-платформенность**:
   - Python работает на всех основных операционных системах: Windows, macOS и Linux.
   - Это означает, что вы можете разрабатывать программу на одной системе, а запускать на другой без изменений в коде.
   - Также Python совместим с популярными облачными платформами, что делает его удобным инструментом для проектов, которые работают на серверах.

5. **Интерактивная среда**:
   - Python предоставляет интерактивную среду, где можно вводить команды и сразу видеть результат их выполнения.
   - Интерпретатор Python позволяет выполнять команды по одной, проверять работу кода в процессе.
   - Инструменты, такие как Jupyter Notebook и Google Colab, позволяют комбинировать текст с программным кодом, создавая подробные учебные материалы или исследовательские отчеты.


## **Области применения Python, инструменты для программирования**

### Области применения Python

1. **Веб-разработка**:
   - Python часто используется для создания веб-сайтов и веб-приложений.
   - Для этого применяются **фреймворки** — наборы инструментов, упрощающих процесс разработки.
   - Примеры фреймворков: Django и Flask.
    - **Django**: полнофункциональный фреймворк для создания сложных веб-приложений с аутентификацией, маршрутизацией, формами и базой данных.
    - **Flask**: более легковесный фреймворк, подходящий для небольших веб-приложений и позволяющий разработчикам добавлять только необходимые компоненты.


2. **Анализ данных и машинное обучение**:
   - Python популярен среди специалистов по анализу данных и искусственному интеллекту благодаря таким библиотекам, как NumPy, pandas, TensorFlow и PyTorch.
   - Python популярен среди специалистов по анализу данных и искусственному интеллекту благодаря таким библиотекам, как:
     - **NumPy**: предоставляет возможности работы с массивами и векторами, а также быстрые математические операции.
     - **pandas**: инструменты для анализа и обработки табличных данных, включая поддержку временных рядов и группировку данных.
     - **TensorFlow**: фреймворк для создания и обучения моделей машинного обучения, особенно в области глубокого обучения.
     - **PyTorch**: еще один фреймворк для создания моделей глубокого обучения, который отличается гибкостью и возможностью использования динамических вычислительных графов.
   - Они помогают обрабатывать большие объемы данных и строить модели машинного обучения для прогнозирования и классификации.

3. **Автоматизация**:
   - Python отлично подходит для автоматизации рутинных задач, таких как сбор данных с веб-сайтов (так называемый **парсинг**), автоматическое тестирование программ и управление операционной системой.
   - Например, с его помощью можно автоматически скачивать отчеты или проверять тысячи страниц на обновления.

4. **Наука и исследования**:
   - Python помогает ученым анализировать данные и визуализировать результаты, используя библиотеки Matplotlib и SciPy для построения графиков и сложных вычислений.

5. **Игровая индустрия**:
   - Разработчики игр могут использовать Python для создания игр, используя инструменты вроде Pygame и Panda3D, которые предоставляют функции для рисования графики и работы с физикой объектов.
  - Известные игры, созданные с использованием Python:
    - **Civilization IV**: стратегическая игра, где Python использовался для искусственного интеллекта и скриптов.
    - **EVE Online**: многопользовательская онлайн-игра, использующая Python для серверной логики.
    - **Battlefield 2**: шутер от первого лица, где Python применялся для настройки игры и управления.


### Инструменты для программирования

1. **Интерпретатор Python**:
   - Чтобы запустить программы, нужно установить интерпретатор Python с официального сайта (https://www.python.org/).
   - Он позволяет вводить и тестировать код прямо в командной строке, а также запускать готовые скрипты.

2. **Редакторы кода**:
   - Чтобы писать код, используют программы, называемые редакторами кода или интегрированными средами разработки (IDE).
   - Примеры редакторов: PyCharm, VSCode, Jupyter Notebook, а также онлайн-среды, такие как Google Colab.

3. **Менеджер пакетов pip**:
   - **pip** — это инструмент для установки дополнительных библиотек и пакетов в Python.
   - Используя команды вроде `pip install`, можно быстро скачать и установить тысячи полезных инструментов из интернета.

4. **Виртуальные окружения**:
   - Виртуальные окружения позволяют создать отдельное место для хранения всех необходимых библиотек и пакетов.
   - Это полезно для разных проектов, когда вам нужно использовать разные версии инструментов или изолировать зависимости проекта.
   - Инструменты virtualenv и venv создают эти изолированные среды для каждого проекта.


## **Практическое задание: Установка и запуск первой программы "Hello, World!" в Google Colab**

### Инструкция по установке Google Colab

1. **Подключение к Google**:
   - Убедитесь, что у вас есть аккаунт Google.
   - Зайдите на https://colab.research.google.com/.

2. **Создание нового ноутбука**:
   - После входа в Google Colab, нажмите кнопку "Файл" → "Новый блокнот".
   - Новый ноутбук откроется в новой вкладке, где вы сможете начать писать и выполнять код.

### Запуск первой программы "Hello, World!"

1. **Добавьте ячейку кода**:
   - В вашем новом ноутбуке будет уже открыта одна ячейка кода.
   - Если нужно добавить дополнительную ячейку, нажмите кнопку "+ Код".

2. **Напишите код**:
   - В ячейке кода введите следующую строку: print("Hello, World!")

3. **Выполните код**:
   - Чтобы выполнить ячейку кода, нажмите на значок "play" (треугольник слева от ячейки) или используйте сочетание клавиш `Shift + Enter`.
   - После выполнения команды внизу ячейки вы увидите надпись "Hello, World!".


In [ ]:
print("Hello, World!")

Hello, World!


**Примечание:**

- **`print`** — встроенная функция в Python, которая выводит текст или данные на экран.
- Аргументы **`print`**:
  - Первый и основной аргумент — это текст или данные, которые мы хотим вывести.
  - Дополнительные параметры позволяют настроить параметры вывода, разделители между элементами (`sep`), окончание строки (`end`) и др.

In [ ]:
print("яблоко", "банан", "вишня", sep=": ")
print("Привет", end="!")

яблоко: банан: вишня
Привет!

Поздравляем! Вы успешно запустили свою первую программу на Python в Google Colab.

In [ ]:
#@title Запуск Нейро Репетитора
tutor_chat_id = 1
# Запуск чата
display_chat_interface(history)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **2. Основы синтаксиса Python**

## **Синтаксис, переменные и типы данных**

### Синтаксис

- **Синтаксис** — это набор правил, определяющих структуру и порядок написания программного кода.
- В Python синтаксис прост и сфокусирован на читаемости кода.
- Основные элементы синтаксиса включают:

1. **Отступы**:
   - Используются для обозначения блоков кода.
   - Стандартным соглашением является 4 пробела на уровень отступа, но главное использовать одинаковое количество пробелов в одном блоке кода..

In [ ]:
if True:  # Блок кода с отступом
    print("Отступ используется для определения блоков кода.")

Отступ используется для определения блоков кода.


2. **Комментарии**:
   - Добавляются с помощью символа `#`.
   - Комментарии поясняют код и не влияют на выполнение программы.

In [ ]:
# Это комментарий, поясняющий код
print("Комментарии начинаются с символа #")

Комментарии начинаются с символа #


3. **Строки**:
   - Строки заключаются в одинарные или двойные кавычки.
   - Многострочные строки можно заключать в тройные кавычки (''' или """).

In [ ]:
single_quoted = 'Одинарные кавычки'
double_quoted = "Двойные кавычки"
multi_line = '''Многострочная
строка'''

### Переменные

- **Переменные** — это именованные области памяти, используемые для хранения данных.
- Основные принципы работы с переменными:

1. **Создание переменной**:
   - Присвоение значения переменной с помощью оператора `=`.

2. **Названия переменных**:
   - Названия переменных чувствительны к регистру, т. е. переменные `age` и `Age` считаются разными.
   - Название переменной должно начинаться с буквы или символа подчеркивания `_`.
   - Название может содержать цифры, но не может начинаться с цифры.
   - Примеры правильных имен: `_counter`, `user_name`, `score2`.

In [ ]:
# Примеры правильных имен переменных
_counter = 5
user_name = "Bob"
score2 = 95

In [ ]:
# Примеры вывода переменных
print("_counter:", _counter)
print("Имя пользователя:", user_name)
print("Оценка:", score2)

_counter: 5
Имя пользователя: Bob
Оценка: 95


In [ ]:
# Переменные чувствительны к регистру
Age = 30
age = 40
print("Значение переменной Age:", Age)
print("Значение переменной age:", age)

Значение переменной Age: 30
Значение переменной age: 40


### Типы данных

1. **Целые числа (int)**:
   - Для хранения целых чисел (положительных или отрицательных).

2. **Вещественные числа (float)**:
   - Для хранения чисел с плавающей запятой.

3. **Строки (str)**:
   - Для хранения текстовой информации.

4. **Булевы значения (bool)**:
   - Хранят значения `True` (истина) или `False` (ложь).

In [ ]:
# Переменные разных типов данных
integer_value = 25          # int
float_value = 1.75          # float
string_value = "Alice"      # str
boolean_value = True        # bool

In [ ]:
# Вывод переменных на экран
print("Целое число:", integer_value)
print("Вещественное число:", float_value)
print("Строка:", string_value)
print("Булево значение:", boolean_value)

Целое число: 25
Вещественное число: 1.75
Строка: Alice
Булево значение: True


In [ ]:
#@title Запуск Нейро Репетитора
tutor_chat_id = 2
# Запуск чата
display_chat_interface(history)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## **Встроенные функции для работы с типами данных, операторы сравнения и логические операторы**

### Встроенные функции для работы с типами данных

Python предоставляет ряд встроенных функций, которые позволяют работать с различными типами данных.

1. **Функция `type`**:
   - Возвращает тип данных переменной.
   - Пример: `type(42)`.

In [ ]:
# Определение типа переменной
integer_value = 42
print("Тип переменной integer_value:", type(integer_value))

Тип переменной integer_value: <class 'int'>


2. **Функции преобразования типов**:
   - Позволяют преобразовывать данные из одного типа в другой.
   - Примеры:
     - `int("42")`: преобразует строку в целое число.
     - `float("3.14")`: преобразует строку в число с плавающей запятой.
     - `str(42)`: преобразует число в строку.
     - `bool(0)`: преобразует число в логическое значение `False`.

In [ ]:
# Пример преобразования строки в число с помощью int
string_number = "42"
integer_number = int(string_number)
print("Строка '42' преобразована в целое число:", integer_number)

Строка '42' преобразована в целое число: 42


In [ ]:
# Пример преобразования строки в число с помощью float
float_number = float("3.14")
print("Строка '3.14' преобразована в число с плавающей запятой:", float_number)

Строка '3.14' преобразована в число с плавающей запятой: 3.14


In [ ]:
# Пример преобразования числа в строку с помощью str
number = 42
number_str = str(number)
print("Преобразованное в строку значение:", number_str)

Преобразованное в строку значение: 42


In [ ]:
# Пример преобразования числа в булево значение с помощью bool
boolean_value = bool(0)
print("Число 0 преобразовано в булево значение:", boolean_value)

Число 0 преобразовано в булево значение: False


**Примечание:**

В Python булевые значения следуют таким правилам:
- 0 (и любые другие значения, оцениваемые как пустые, например, None, пустой список или пустая строка) преобразуются в False.
- 1 (и любые ненулевые числа или непустые объекты) преобразуются в True.

In [ ]:
#@title Запуск Нейро Репетитора
tutor_chat_id = 3
# Запуск чата
display_chat_interface(history)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Операторы сравнения

Операторы сравнения позволяют сравнивать значения и возвращать логическое значение (`True` или `False`).

1. **Равно (`==`)**:
   - Проверяет, равны ли два значения.
   - Пример: `5 == 5` возвращает `True`.

2. **Не равно (`!=`)**:
   - Проверяет, не равны ли два значения.
   - Пример: `5 != 3` возвращает `True`.

3. **Больше (`>`) и меньше (`<`)**:
   - Сравнивают два значения, проверяя, больше или меньше ли первое значение второго.
   - Пример: `10 > 5` возвращает `True`, а `3 < 1` возвращает `False`.

4. **Больше или равно (`>=`) и меньше или равно (`<=`)**:
   - Проверяют, больше/меньше ли значение либо равно ему.
   - Пример: `10 >= 10` возвращает `True`, а `3 <= 2` возвращает `False`.


In [ ]:
# Примеры операторов сравнения
print("5 == 5:", 5 == 5)
print("5 != 3:", 5 != 3)
print("10 > 5:", 10 > 5)
print("3 < 1:", 3 < 1)
print("10 >= 10:", 10 >= 10)
print("3 <= 2:", 3 <= 2)

5 == 5: True
5 != 3: True
10 > 5: True
3 < 1: False
10 >= 10: True
3 <= 2: False


### Логические операторы

Логические операторы позволяют комбинировать несколько условий.

1. **and**:
   - Возвращает `True`, если оба условия истинны.
   - Пример: `(5 > 2) and (3 != 4)` возвращает `True`.

2. **or**:
   - Возвращает `True`, если хотя бы одно условие истинно.
   - Пример: `(5 > 10) or (3 != 4)` возвращает `True`.

3. **not**:
   - Изменяет значение условия на противоположное.
   - Пример: `not (5 == 5)` возвращает `False`.

In [ ]:
# Примеры логических операторов
print("(5 > 2) and (3 != 4):", (5 > 2) and (3 != 4))
print("(5 > 10) or (3 != 4):", (5 > 10) or (3 != 4))
print("not (5 == 5):", not (5 == 5))

(5 > 2) and (3 != 4): True
(5 > 10) or (3 != 4): True
not (5 == 5): False


In [ ]:
#@title Запуск Нейро Репетитора
tutor_chat_id = 4
# Запуск чата
display_chat_interface(history)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## **Практическое задание: Создать программу с арифметическими операциями**

### Задание

В этом задании создадим программу, которая будет выполнять базовые арифметические операции (сложение, вычитание, умножение и деление) между двумя числами. Программа также должна выводить результаты на экран.

**Задачи**:

1. Введите два числа.

2. Выполните арифметические операции:
   - Сложение (`+`).
   - Вычитание (`-`).
   - Умножение (`*`).
   - Деление (`/`).
   - Целочисленное деление (`//`).
   - Деление с остатком (`%`).
   - Возведение в степень (`**`).

3. Выведите результаты каждой операции.

### Решение

In [ ]:
# Вводим два числа, преобразуем их к типу float
number1 = float(input("Введите первое число: "))
number2 = float(input("Введите второе число: "))

Введите первое число: 9
Введите второе число: 4


In [ ]:
# Выполняем арифметические операции
addition = number1 + number2  # Сложение
subtraction = number1 - number2  # Вычитание
multiplication = number1 * number2  # Умножение
division = number1 / number2  # Деление
integer_division = number1 // number2  # Целочисленное деление
remainder_division = number1 % number2  # Деление с остатком
power = number1 ** number2  # Возведение в степень

In [ ]:
# Выводим результаты на экран с указанием операции
print("Результаты арифметических операций:")
print(f"Сложение: {number1} + {number2} = {addition}")
print(f"Вычитание: {number1} - {number2} = {subtraction}")
print(f"Умножение: {number1} * {number2} = {multiplication}")
print(f"Деление: {number1} / {number2} = {division}")
print(f"Целочисленное деление: {number1} // {number2} = {integer_division}")
print(f"Деление с остатком: {number1} % {number2} = {remainder_division}")
print(f"Возведение в степень: {number1} ** {number2} = {power}")

Результаты арифметических операций:
Сложение: 9.0 + 4.0 = 13.0
Вычитание: 9.0 - 4.0 = 5.0
Умножение: 9.0 * 4.0 = 36.0
Деление: 9.0 / 4.0 = 2.25
Целочисленное деление: 9.0 // 4.0 = 2.0
Деление с остатком: 9.0 % 4.0 = 1.0
Возведение в степень: 9.0 ** 4.0 = 6561.0


**Примечание:**
- **`input`** - встроенная функция, которая используется для ввода данных от пользователя. Возвращает введенное значение как строку, поэтому его нужно преобразовать в нужный тип данных с помощью функций преобразования (`float`, `int` и др.).

- **Целочисленное деление (`//`)** - операция, которая возвращает только целую часть от деления.
   - Например, `9 // 4` возвращает `2`, так как 9 разделить на 4 дает 2 с остатком.

- **Деление с остатком (`%`)** - операция, которая возвращает остаток от деления.
   - Например, `9 % 4` возвращает `1`, так как 9 делится на 4 с остатком 1.

In [ ]:
#@title Запуск Нейро Репетитора
tutor_chat_id = 5
# Запуск чата
display_chat_interface(history)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **3. Условия**

## **Конструкции `if-else`**

Позволяют выполнять различные блоки кода в зависимости от выполнения определенных условий.

1. **`if`**:
   - Проверяет логическое условие.
   - Если условие истинно (`True`), выполняется блок кода, связанный с `if`.

2. **`else`**:
   - Выполняется, если условие в блоке `if` ложно (`False`).
   - `else` не принимает условий, а просто указывает альтернативный блок кода.

3. **`elif`** (сокращение от "else if"):
   - Позволяет проверять дополнительные условия, если первое условие в `if` не было выполнено.
   - В отличие от `else`, `elif` принимает условия.

### Общая структура `if-else`

```python
if условие:
    # блок кода, если условие истинно
elif другое_условие:
    # блок кода, если другое условие истинно
else:
    # блок кода, если ни одно из условий не было выполнено
    

### Примеры использования

In [ ]:
# Вводим число
number = int(input("Введите число: "))

# Проверяем, положительное ли число
if number > 0:
    print("Число положительное.")

# Проверяем, отрицательное ли число
elif number < 0:
    print("Число отрицательное.")

# Если число не положительное и не отрицательное, значит, оно равно нулю
else:
    print("Число равно нулю.")

Введите число: 0
Число равно нулю.


In [ ]:
# Вводим возраст
age = int(input("Введите ваш возраст: "))

# Если возраст меньше 18 лет
if age < 18:
    print("Вы несовершеннолетний.")

# Если возраст меньше 65 лет
elif age < 65:
    print("Вы взрослый.")

# Если возраст 65 и старше
else:
    print("Вы пенсионер.")

Введите ваш возраст: 66
Вы пенсионер.


## **Вложенные условия**

**Вложенные условия** — это конструкции `if-else`, находящиеся внутри других условий.

- Позволяют создавать более сложную логику, когда внутри одного условия проверяются дополнительные подусловия.
- Такие конструкции могут использоваться для выполнения различных блоков кода на основании нескольких уровней проверок.



### Общая структура вложенных условий

```python
if условие_1:
    # блок кода, если условие_1 истинно
    if подусловие_1:
        # блок кода, если подусловие_1 истинно
    else:
        # блок кода, если подусловие_1 ложно
else:
    # блок кода, если условие_1 ложно
    

### Примеры использования

In [ ]:
# Вводим число
number = int(input("Введите число: "))

Введите число: 10


In [ ]:
# Внешнее условие проверяет, положительное ли число
if number > 0:
    print("Число положительное.")
    # Внутреннее условие проверяет, четное ли число
    if number % 2 == 0:
        print("Число четное.")
    else:
        print("Число нечетное.")

# Если не положительное, внешнее условие проверяет, отрицательное ли число
elif number < 0:
    print("Число отрицательное.")
    # Внутреннее условие проверяет, кратно ли число 5
    if number % 5 == 0:
        print("Число кратно 5.")
    else:
        print("Число не кратно 5.")

# Если число не положительное и не отрицательное, оно равно нулю
else:
    print("Число равно нулю.")

Число положительное.
Число четное.


In [ ]:
#@title Запуск Нейро Репетитора
tutor_chat_id = 6
# Запуск чата
display_chat_interface(history)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## **Практическое задание: Программа проверки допустимости водить автомобиль**

### Задание

Создайте программу, которая будет проверять допустимость вождения автомобиля в зависимости от возраста и наличия водительского удостоверения.

**Условия**:

1. Если возраст меньше 16 лет, вождение недопустимо.
2. Если возраст 16 лет и старше:
   - Если у пользователя есть водительское удостоверение, вождение допустимо.
   - Если удостоверения нет, вождение недопустимо.

### Решение

In [ ]:
# Запрашиваем у пользователя возраст
age = int(input("Введите ваш возраст: "))

Введите ваш возраст: 25


In [ ]:
# Проверяем допустимость вождения на основе возраста и наличия удостоверения
if age < 16:
    print("Вам запрещено водить автомобиль.")
else:
    # Запрашиваем у пользователя наличие водительского удостоверения
    has_license = input("У вас есть водительское удостоверение? (да/нет): ").strip().lower()

    # Вложенное условие для проверки наличия удостоверения
    if has_license == "да":
        print("Вы можете водить автомобиль.")
    else:
        print("Вам запрещено водить автомобиль без водительского удостоверения.")

У вас есть водительское удостоверение? (да/нет): да
Вы можете водить автомобиль.


**Примечание:**
  - Метод **`strip()`** удаляет пробелы и другие незначащие символы в начале и в конце строки, что полезно при вводе данных.
  - Метод **`lower()`** преобразует все символы строки в нижний регистр, делая ввод нечувствительным к регистру.


In [ ]:
#@title Запуск Нейро Репетитора
tutor_chat_id = 7
# Запуск чата
display_chat_interface(history)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **4. Циклы: for и while. Операторы `break` и `continue`**

## Циклы for и while

**Циклы** позволяют повторять выполнение определенного блока кода несколько раз, что дает возможность автоматически обрабатывать множество элементов или выполнять задачи с заранее неизвестным количеством итераций.

### **Цикл for**

- Используется для прохода по элементам коллекции, таким как список, строка или диапазон чисел, и выполнения заданного блока кода для каждого элемента.
- **Общая структура**:

```python
for элемент в последовательности:
    # блок кода, который выполняется для каждого элемента


**Пояснение:**
- элемент — переменная, которая будет принимать значение каждого элемента в последовательности.
- последовательность — это коллекция элементов (например, список, строка или диапазон чисел), по которым будет проходить цикл.

Особенности цикла for:

- Используется для итерации по заранее определенной последовательности элементов.

### Примеры использования

In [ ]:
# Цикл for: проходим по элементам списка
numbers = [1, 2, 3, 4, 5]  # Инициализируем список чисел
print("Цикл for:")
for number in numbers:  # Проходим по каждому элементу списка
    print(number)  # Выводим текущее число из списка

Цикл for:
1
2
3
4
5


In [ ]:
# Цикл for: проходим по символам строки
text = "Python"  # Инициализируем строку
print("Цикл for (по строке):")
for char in text:  # Проходим по каждому символу строки
    print(char)  # Выводим текущий символ из строки

Цикл for (по строке):
P
y
t
h
o
n


### **Цикл while**

- Повторяет выполнение блока кода до тех пор, пока выполняется определенное логическое условие.
- **Общая структура**:

```python
while условие:
    # блок кода, который выполняется, пока условие истинно


Пояснение:
- условие — это выражение, которое проверяется перед каждой итерацией цикла.
- Пока условие истинно (True), цикл будет повторяться.

Особенности цикла while:
- Подходит для задач с заранее неизвестным количеством итераций.

### Примеры использования

In [ ]:
# Цикл while: повторяем выполнение кода, пока счетчик не станет равным нулю
counter = 5  # Инициализируем счетчик значением 5
print("Цикл while:")
while counter > 0:  # Продолжаем выполнение цикла, пока значение счетчика больше 0
    print(f"Счетчик: {counter}")  # Выводим текущее значение счетчика
    counter -= 1  # Уменьшаем значение счетчика на 1 при каждой итерации


Цикл while:
Счетчик: 5
Счетчик: 4
Счетчик: 3
Счетчик: 2
Счетчик: 1


In [ ]:
# Цикл while: выполняется, пока не введен правильный пароль
password = "qwerty"  # Устанавливаем правильный пароль
attempt = ""  # Переменная для попыток ввода пароля
print("\nЦикл while (проверка пароля):")
while attempt != password:  # Цикл продолжается, пока введенный пароль не совпадает с правильным
    attempt = input("Введите пароль: ")  # Получаем ввод пользователя
print("Пароль введен правильно.")  # Сообщение выводится после правильного ввода пароля


Цикл while (проверка пароля):
Введите пароль: 11111
Введите пароль: qqqqq
Введите пароль: qwerty
Пароль введен правильно.


## **Операторы `break` и `continue`**

### **Оператор break**

- Прерывает выполнение текущего цикла и выводит программу из цикла досрочно.
- Используется, когда нужно немедленно остановить цикл при выполнении определенного условия.
- **Общая структура**:

```python
for элемент в последовательности:
    if условие:
        break  # завершает выполнение цикла
    # блок кода, который выполняется, пока не выполнено условие break


### Пример использования

In [ ]:
# Пример оператора break в цикле for
print("Пример break:")  # Заголовок для вывода
for number in range(1, 11):  # Цикл for проходит по числам от 1 до 10
    if number == 5:  # Проверяем, равно ли текущее число 5
        break  # Прерываем выполнение цикла, если число равно 5
    print(number)  # Выводим текущее число, если условие выше не выполнено


Пример break:
1
2
3
4


### **Оператор continue**

- Прерывает текущую итерацию цикла и переходит к следующей итерации.
- Позволяет пропустить оставшийся код текущей итерации и начать новую.
- **Общая структура**:

```python
while условие:
    if другое_условие:
        continue  # пропускает оставшийся код в текущей итерации
    # блок кода, который выполняется, если не выполнено условие continue


### Примеры использования

In [ ]:
# Пример оператора continue в цикле while
print("\nПример continue:")  # Заголовок для вывода
counter = 0  # Инициализируем счетчик значением 0
while counter < 10:  # Выполняем цикл, пока счетчик меньше 10
    counter += 1  # Увеличиваем значение счетчика на 1
    if counter % 2 == 0:  # Проверяем, является ли значение четным
        continue  # Пропускаем оставшийся код в текущей итерации, если значение четное
    print(counter)  # Выводим значение счетчика, если условие выше не выполнено


Пример continue:
1
3
5
7
9


In [ ]:
#@title Запуск Нейро Репетитора
tutor_chat_id = 8
# Запуск чата
display_chat_interface(history)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## **Практическое задание: Программа для расчета суммы четных чисел**

### Задание

Создайте программу, которая будет принимать диапазон чисел и вычислять сумму всех четных чисел в этом диапазоне.

**Задачи**:

1. Введите два числа, обозначающих начало и конец диапазона.
2. Используйте цикл `for` для прохождения по каждому числу в диапазоне.
3. Пропустите нечетные числа с помощью оператора `continue`.
4. Посчитайте сумму всех четных чисел и выведите результат.

### Решение

In [ ]:
# Вводим начало и конец диапазона
start = int(input("Введите начало диапазона: "))
end = int(input("Введите конец диапазона: "))

Введите начало диапазона: 2
Введите конец диапазона: 11


In [ ]:
# Инициализируем переменную для хранения суммы
even_sum = 0

# Используем цикл for для прохождения по диапазону
for number in range(start, end + 1):
    if number % 2 != 0:
        continue  # Пропускаем нечетные числа
    even_sum += number  # Добавляем четное число к сумме

# Выводим результат
print(f"Сумма всех четных чисел в диапазоне от {start} до {end} составляет: {even_sum}")

Сумма всех четных чисел в диапазоне от 2 до 11 составляет: 30


**Примечание:**
  - **Встроенная функция `range`** возвращает последовательность чисел от начального (`start`) до конечного (`end`) значения, но не включает значение `end`.
  - Чтобы включить конечное значение, нужно указать `end + 1`.


In [ ]:
#@title Запуск Нейро Репетитора
tutor_chat_id = 9
# Запуск чата
display_chat_interface(history)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## **Заключение**

В этом уроке мы познакомились с базовыми структурами языка Python. Рассмотрели основные элементы синтаксиса, переменные и типы данных, изучили принципы работы с условиями (`if-else`) и циклами (`for` и `while`). Также разобрали такие инструменты управления, как операторы `break` и `continue`.

В ходе практических заданий мы:
- Запустили первую программу "Hello, World!" в Google Colab.
- Создали программу для выполнения арифметических операций.
- Проверили навыки с условиями в программе, определяющей допустимость водить автомобиль.
- Реализовали задачу по расчёту суммы четных чисел в заданном диапазоне с использованием циклов.

### Полезные материалы:

- [Документация по Python](https://docs.python.org/3/)
- [PEP 8 — Руководство по стилю написания кода на Python](https://peps.python.org/pep-0008/)
- [Google Colab: руководство для начинающих](https://research.google.com/colaboratory/)

Эти ресурсы помогут вам углубить свои знания о языке и писать более эффективный, чистый и понятный код

In [ ]:
#@title Запуск Нейро Репетитора
tutor_chat_id = 10
# Запуск чата
display_chat_interface(history)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **Вопросы для проверки**

In [ ]:
#@title Вопросы для проверки знаний

import pandas as pd
import random
import requests

# @title Получение таблицы экзаменатора

def download_test_file():
    url = "http://62.113.108.33:80/vebinar-v1/get-test"
    headers = {"Content-Type": "application/json",
        "Authorization": '89e768ed-4bc1-4e12-9139-45023e487b78'
        }
    response = requests.get(url, headers=headers)

    # Сохранение файла
    if response.status_code == 200:
        # Получаем имя файла из Content-Disposition, удаляя кавычки
        content_disposition = response.headers.get('content-disposition')
        if content_disposition:
            filename = content_disposition.split('filename=')[-1].strip('"')
            with open(filename, 'wb') as file:
                file.write(response.content)
            print(f"Данные экзаменатора успешно загружены.")
            return True
        else:
            print("Не удалось получить имя файла из заголовка Content-Disposition.")
            return False
    else:
        print("Ошибка при загрузке данных:", response.status_code)
        print(response.text)
        return False

# Тема занятия
lesson_topic = 'Основы Python вебинар'

# Загрузка данных для экзаменатора
check_test_file = download_test_file()
if check_test_file:
    # Чтение данных из CSV файла
    df = pd.read_csv('/content/Questions.csv')

    # Преобразование колонки "Правильный ответ" в числа
    df['Правильный ответ'] = pd.to_numeric(df['Правильный ответ'], errors='coerce')

    # Фильтрация данных по теме занятия
    filtered_df = df[df['Занятие'] == lesson_topic]

    # Проверка, что достаточно вопросов для выбора
    if len(filtered_df) < 6:
        print("Недостаточно вопросов для выбора 6 случайных вопросов.")
    else:
        # Выбор случайных 6 вопросов
        random_indices = random.sample(range(len(filtered_df)), 6)

        # Инициализация переменной для подсчета правильных ответов
        correct_answers = 0

        # Предоставление вопросов и проверка ответов
        for idx in random_indices:
            question = filtered_df.iloc[idx]['Вопрос']
            correct_answer = filtered_df.iloc[idx]['Правильный ответ']
            comment = filtered_df.iloc[idx]['Комментарий']
            print()
            print(question)
            print()
            # Получение ответа пользователя
            user_answer = input("Введите ваш ответ: ").strip()

            # Преобразование ответа пользователя в число
            try:
                user_answer = int(user_answer)
            except ValueError:
                print(f"\nНеправильный формат ответа. Правильный ответ: {correct_answer}")
                print(f"Комментарий: {comment}")
                continue

            # Проверка правильности ответа
            if user_answer == correct_answer:
                correct_answers += 1
                print("\nПравильно!")
            else:
                print(f"\nНеправильно. Правильный ответ: {correct_answer}")
                print(f"Комментарий: {comment}")

        # Вывод результата количества правильных ответов и общего количества вопросов
        print(f"\nПравильно {correct_answers} из 6")
else:
    print('Исправьте ошибку и попробуете снова!')


Данные экзаменатора успешно загружены.

7. Какой из этих инструментов не относится к среде разработки Python? 
1) PyCharm 
2) VSCode 
3) Excel

Введите ваш ответ: 3

Правильно!

6. Для чего в Python используется Django? 
1) Машинное обучение 
2) Веб-разработка 
3) Автоматизация тестирования

Введите ваш ответ: 2

Правильно!

20. Что делает оператор `break` в цикле Python? 
1) Пропускает текущую итерацию 
2) Прерывает выполнение цикла 
3) Ничего не делает

Введите ваш ответ: 1

Неправильно. Правильный ответ: 2
Комментарий: Ответ '2' является правильным для вопроса: '20. Что делает оператор `break` в цикле Python? 1) Пропускает текущую итерацию 2) Прерывает выполнение цикла 3) Ничего не делает', потому что оператор `break` в Python используется для немедленного прерывания выполнения цикла, прекращая его выполнение и переходя к следующему после цикла коду.

2. В каком году был впервые выпущен Python? 
1) 1989 год 
2) 1991 год 
3) 1995 год

Введите ваш ответ: 2

Правильно!

8. Какая функци

# **Домашнее задание**

Выполните задание https://colab.research.google.com/drive/1rQ0hWvY1nXRw9RbKp5SXrajlj3jNa1Wm?usp=sharing и сохраните в файле с расширением .ipynb

# **Проверка домашнего задания**
(без ошибок но с незапущенными ячейками) https://colab.research.google.com/drive/1Ebz-2mhJFQtmzJWeSbWHIzwLXaMVVrBC?usp=sharing

In [ ]:
#@title Выбираем файл с выполненным домашним заданием
from google.colab import files

# Загружаем файлы
uploaded = files.upload()

# Перебираем все загруженные файлы
for filename in uploaded.keys():
    print('Файл "{name}" загружен!'.format(name=filename))

    # Получение и вывод информации о ноутбуке
    resheniye = process_notebook(filename, output_directory)


Saving ДЗ_РЕШЕНИЕ_Python_на_ChatGPT_День_1 (1).ipynb to ДЗ_РЕШЕНИЕ_Python_на_ChatGPT_День_1 (1) (3).ipynb
Файл "ДЗ_РЕШЕНИЕ_Python_на_ChatGPT_День_1 (1) (3).ipynb" загружен!


In [ ]:
#@title Запуск проверки
import json
import requests
from IPython.core.display import display, HTML, Markdown

tema = "Основы Python"
dz = '''Задачи уровня lite:
1. Простой калькулятор:

Напишите программу, которая запрашивает у пользователя два числа и выполняет их сложение. Выведите результат на экран.
2. Проверка четности числа:

Запросите у пользователя число и выведите на экран, является ли это число четным или нечетным.
3. Подсчет количества слов:

Напишите программу, которая просит пользователя ввести предложение и подсчитывает количество слов в этом предложении.
4. Проверка на большее число:

Напишите программу, которая запрашивает у пользователя два числа и выводит на экран большее из них. Если числа равны, программа должна сообщить, что числа равны.
5. Сумма чисел до N:

Напишите программу, которая просит пользователя ввести положительное число N, а затем вычисляет сумму всех чисел от 1 до N.
Задачи уровня pro:
6. Калькулятор времени:

Напишите программу, которая запрашивает у пользователя количество секунд и преобразует их в часы, минуты и секунды. Выведите результат в формате "ЧЧ:ММ:СС".
7. Расчет стоимости покупки:

Напишите программу, которая запрашивает у пользователя стоимость одного товара и количество товаров, которое он хочет купить. Затем вычислите и выведите общую стоимость покупки.
8. Конвертер валют:

Напишите программу, которая запрашивает у пользователя сумму в долларах и конвертирует её в евро по текущему курсу (курс можно задать в программе как константу). Выведите результат конвертации.
9. Проверка делимости чисел:

Напишите программу, которая запрашивает у пользователя два числа и проверяет, делится ли первое число на второе без остатка. Программа должна выводить соответствующее сообщение о результате.
10. Определение сезона по месяцу:

Напишите программу, которая запрашивает у пользователя номер месяца (от 1 до 12) и выводит, к какому сезону этот месяц относится. Программа должна корректно обрабатывать неправильный ввод (например, если пользователь введет число за пределами диапазона).'''

url = "http://62.113.108.33/vebinar-v1/check_dz"
headers = {"Content-Type": "application/json",
    "Authorization": '89e768ed-4bc1-4e12-9139-45023e487b78'
    }
data = {
    "tema": tema,
    "dz": dz,
    "resheniye": resheniye
}

response = requests.post(url, headers=headers, data=json.dumps(data))
if response.status_code == 200:
  display(Markdown(json.loads(response.text)['choices'][0]['message']['content']))
  # print(json.loads(response.text)['choices'][0]['message']['content'])
else:
  print("Возникла ошибка:", response.status_code)
  print(response.text)

### Оценка решения студента

#### Общие замечания:
1. **Запуск ячеек**: Все ячейки в решении не были запущены. Это важно, так как выполнение кода позволяет проверить его работоспособность и корректность.
2. **Форматирование**: Использование markdown для описания задач и комментариев в коде выполнено хорошо, что делает решение структурированным и понятным.

#### Индивидуальные задачи:

1. **Простой калькулятор**:
   - Код корректен и выполняет задачу.
   - Результат: 17.0 (верно).

2. **Проверка четности числа**:
   - Код корректен и выполняет задачу.
   - Результат: Число 8 является четным (верно).

3. **Подсчет количества слов**:
   - Код корректен и выполняет задачу.
   - Результат: Количество слов в предложении: 4 (верно).

4. **Проверка на большее число**:
   - Код корректен и выполняет задачу.
   - Результат: Большее число: 12.0 (верно).

5. **Сумма чисел до N**:
   - Код корректен и выполняет задачу.
   - Результат: Сумма чисел от 1 до 5 : 15 (верно).

6. **Калькулятор времени**:
   - Код корректен и выполняет задачу.
   - Результат: Время: 00:45:00 (верно).

7. **Расчет стоимости покупки**:
   - Код корректен и выполняет задачу.
   - Результат: Общая стоимость покупки: 2940000.0 (верно).

8. **Конвертер валют**:
   - Код корректен и выполняет задачу.
   - Результат: Сумма в евро: 119.0 (верно).

9. **Проверка делимости чисел**:
   - Код корректен и выполняет задачу.
   - Результат: 34 делится на 17 без остатка (верно).

10. **Определение сезона по месяцу**:
    - Код корректен и выполняет задачу.
    - Результат: Месяц номер 7 относится к сезону лето (верно).

#### Рекомендации:
1. **Запуск кода**: Обязательно запускайте все ячейки перед сдачей работы, чтобы убедиться в корректности выполнения.
2. **Проверка ввода**: В задачах, где требуется ввод от пользователя, можно добавить дополнительные проверки на корректность ввода (например, проверка на ввод чисел в задачах 1, 2, 4, 5, 7, 9, 10).
3. **Форматирование вывода**: В некоторых задачах можно улучшить форматирование вывода для большей читаемости (например, в задаче 6 можно добавить пробелы между часами, минутами и секундами).

#### Заключение:
Студент справился с домашним заданием, все задачи выполнены корректно. Однако, необходимо запускать все ячейки перед сдачей работы.

#### Оценка:
8/10

- 2 балла снято за незапущенные ячейки.

# **Проверка домашнего задания**
(с двумя ошибками) https://colab.research.google.com/drive/16LksX2wx42Vg-FTUduUp0qLRLGf2W_-V?usp=sharing

In [ ]:
#@title Выбираем файл с выполненным домашним заданием
from google.colab import files

# Загружаем файлы
uploaded = files.upload()

# Перебираем все загруженные файлы
for filename in uploaded.keys():
    print('Файл "{name}" загружен!'.format(name=filename))

    # Получение и вывод информации о ноутбуке
    resheniye = process_notebook(filename, output_directory)


Saving ДЗ_РЕШЕНИЕ_с_2_ошибками_Python_на_ChatGPT_День_1.ipynb to ДЗ_РЕШЕНИЕ_с_2_ошибками_Python_на_ChatGPT_День_1.ipynb
Файл "ДЗ_РЕШЕНИЕ_с_2_ошибками_Python_на_ChatGPT_День_1.ipynb" загружен!


In [ ]:
#@title Запуск проверки
import json
import requests
from IPython.core.display import display, HTML, Markdown

tema = "Основы Python"
dz = '''Задачи уровня lite:
1. Простой калькулятор:

Напишите программу, которая запрашивает у пользователя два числа и выполняет их сложение. Выведите результат на экран.
2. Проверка четности числа:

Запросите у пользователя число и выведите на экран, является ли это число четным или нечетным.
3. Подсчет количества слов:

Напишите программу, которая просит пользователя ввести предложение и подсчитывает количество слов в этом предложении.
4. Проверка на большее число:

Напишите программу, которая запрашивает у пользователя два числа и выводит на экран большее из них. Если числа равны, программа должна сообщить, что числа равны.
5. Сумма чисел до N:

Напишите программу, которая просит пользователя ввести положительное число N, а затем вычисляет сумму всех чисел от 1 до N.
Задачи уровня pro:
6. Калькулятор времени:

Напишите программу, которая запрашивает у пользователя количество секунд и преобразует их в часы, минуты и секунды. Выведите результат в формате "ЧЧ:ММ:СС".
7. Расчет стоимости покупки:

Напишите программу, которая запрашивает у пользователя стоимость одного товара и количество товаров, которое он хочет купить. Затем вычислите и выведите общую стоимость покупки.
8. Конвертер валют:

Напишите программу, которая запрашивает у пользователя сумму в долларах и конвертирует её в евро по текущему курсу (курс можно задать в программе как константу). Выведите результат конвертации.
9. Проверка делимости чисел:

Напишите программу, которая запрашивает у пользователя два числа и проверяет, делится ли первое число на второе без остатка. Программа должна выводить соответствующее сообщение о результате.
10. Определение сезона по месяцу:

Напишите программу, которая запрашивает у пользователя номер месяца (от 1 до 12) и выводит, к какому сезону этот месяц относится. Программа должна корректно обрабатывать неправильный ввод (например, если пользователь введет число за пределами диапазона).'''

url = "http://62.113.108.33/vebinar-v1/check_dz"
headers = {"Content-Type": "application/json",
    "Authorization": '89e768ed-4bc1-4e12-9139-45023e487b78'
    }
data = {
    "tema": tema,
    "dz": dz,
    "resheniye": resheniye
}

response = requests.post(url, headers=headers, data=json.dumps(data))
if response.status_code == 200:
  display(Markdown(json.loads(response.text)['choices'][0]['message']['content']))
  # print(json.loads(response.text)['choices'][0]['message']['content'])
else:
  print("Возникла ошибка:", response.status_code)
  print(response.text)

### Оценка решения студента

#### **Задачи уровня lite:**

**1. Простой калькулятор:**
- Код:
```python
num1 = float(input("Введите первое число: "))
num2 = float(input("Введите второе число: "))
result = num1 + num2
print("Результат сложения:", result)
```
- Оценка: 10/10
- Комментарий: Решение корректное, работает правильно. 

**2. Проверка четности числа:**
- Код:
```python
number = int(input("Введите число: "))
if number % 2 == 0:
    print("Число", number, "является четным.")
else:
    print("Число", number, "является нечетным.")
```
- Оценка: 10/10
- Комментарий: Решение корректное, работает правильно.

**3. Подсчет количества слов:**
- Код:
```python
sentence = input("Введите предложение: ")
words = sentence.split()
word_count = len(words)
print("Количество слов в предложении:", word_count)
```
- Оценка: 10/10
- Комментарий: Решение корректное, работает правильно.

**4. Проверка на большее число:**
- Код:
```python
num1 = float(input("Введите первое число: "))
num2 = float(input("Введите второе число: "))
if num1 > num2:
    print("Большее число:", num1)
elif num2 > num1:
    print("Большее число:", num2)
else:
    print("Числа равны.")
```
- Оценка: 10/10
- Комментарий: Решение корректное, работает правильно.

**5. Сумма чисел до N:**
- Код:
```python
N = int(input("Введите положительное число N: "))
if N <= 0:
    print("Ошибка: Введено не положительное число.")
else:
    sum_of_numbers = 0
    for i in range(1, N + 1):
        sum_of_numbers += i
    print("Сумма чисел от 1 до", N, ":", sum_of_numbers)
```
- Оценка: 10/10
- Комментарий: Решение корректное, работает правильно.

#### **Задачи уровня pro:**

**6. Калькулятор времени:**
- Код:
```python
total_seconds = int(input("Введите количество секунд: "))
hours = total_seconds // 3600
remaining_seconds = total_seconds % 3600
minutes = remaining_seconds // 60
seconds = remaining_seconds % 60
print(f"Время: {hours:02d}:{minutes:02d}:{seconds:02d}")
```
- Оценка: 10/10
- Комментарий: Решение корректное, работает правильно.

**7. Расчет стоимости покупки:**
- Код:
```python
price_per_item = float(input("Введите стоимость одного товара: "))
quantity = int(input("Введите количество товаров: "))
total_cost = price_per_item * quantity
print("Общая стоимость покупки:", total_cost)
```
- Оценка: 10/10
- Комментарий: Решение корректное, работает правильно.

**8. Конвертер валют:**
- Код:
```python
exchange_rate = 0.85
amount_in_dollars = float(input("Введите сумму в долларах: "))
amount_in_euros = amount_in_dollars * exchange_rate
print("Сумма в евро:", amount_in_euros)
```
- Оценка: 10/10
- Комментарий: Решение корректное, работает правильно.

**9. Проверка делимости чисел:**
- Код:
```python
num1 = int(input("Введите первое число: "))
num2 = int(input("Введите второе число: "))
if num1 % num2 == 1:
    print(f"{num1} делится на {num2} без остатка")
else:
    print(f"{num1} не делится на {num2} без остатка")
```
- Оценка: 8/10
- Комментарий: В условии проверки делимости ошибка. Правильное условие должно быть `if num1 % num2 == 0:`.

**10. Определение сезона по месяцу:**
- Код:
```python
month = int(input("Введите номер месяца (от 1 до 12): "))
if month == 12 or month <= 1:
    print("Зима")
elif month >= 3 and month <= 5:
    print("Весна")
elif month >= 6 and month <= 8:
    print("Лето")
elif month >= 9 and month <= 11:
    print("Осень")
else:
    print("Ошибка: введите число от 1 до 12")
```
- Оценка: 8/10
- Комментарий: Ошибка в условии для зимы. Правильное условие должно быть `if month == 12 or month <= 2:`.

### Заключение
Студент справился с большинством задач, однако в задачах 9 и 10 есть небольшие ошибки в логике условий. В остальном, решение корректное и работает правильно.

### Итоговая оценка: 9/10

### Рекомендации:
1. Внимательно проверяйте условия в логических выражениях.
2. Тестируйте код на различных входных данных, чтобы убедиться в его корректности.
3. Продолжайте практиковаться и улучшать свои навыки программирования.

# **Проверка домашнего задания**
(много ошибок) https://colab.research.google.com/drive/1j8pgGWKdD82lLrnra1pAhTLI2eJ2m5Ew?usp=sharing

In [ ]:
#@title Выбираем файл с выполненным домашним заданием
from google.colab import files

# Загружаем файлы
uploaded = files.upload()

# Перебираем все загруженные файлы
for filename in uploaded.keys():
    print('Файл "{name}" загружен!'.format(name=filename))
    # Получение и вывод информации о ноутбуке
    resheniye = process_notebook(filename, output_directory)



Saving ДЗ_РЕШЕНИЕ_много_ошибок_Python_на_ChatGPT_День_1.ipynb to ДЗ_РЕШЕНИЕ_много_ошибок_Python_на_ChatGPT_День_1.ipynb
Файл "ДЗ_РЕШЕНИЕ_много_ошибок_Python_на_ChatGPT_День_1.ipynb" загружен!


In [ ]:
#@title Запуск проверки
import json
import requests
from IPython.core.display import display, HTML, Markdown

tema = "Основы Python"
dz = '''Задачи уровня lite:
1. Простой калькулятор:

Напишите программу, которая запрашивает у пользователя два числа и выполняет их сложение. Выведите результат на экран.
2. Проверка четности числа:

Запросите у пользователя число и выведите на экран, является ли это число четным или нечетным.
3. Подсчет количества слов:

Напишите программу, которая просит пользователя ввести предложение и подсчитывает количество слов в этом предложении.
4. Проверка на большее число:

Напишите программу, которая запрашивает у пользователя два числа и выводит на экран большее из них. Если числа равны, программа должна сообщить, что числа равны.
5. Сумма чисел до N:

Напишите программу, которая просит пользователя ввести положительное число N, а затем вычисляет сумму всех чисел от 1 до N.
Задачи уровня pro:
6. Калькулятор времени:

Напишите программу, которая запрашивает у пользователя количество секунд и преобразует их в часы, минуты и секунды. Выведите результат в формате "ЧЧ:ММ:СС".
7. Расчет стоимости покупки:

Напишите программу, которая запрашивает у пользователя стоимость одного товара и количество товаров, которое он хочет купить. Затем вычислите и выведите общую стоимость покупки.
8. Конвертер валют:

Напишите программу, которая запрашивает у пользователя сумму в долларах и конвертирует её в евро по текущему курсу (курс можно задать в программе как константу). Выведите результат конвертации.
9. Проверка делимости чисел:

Напишите программу, которая запрашивает у пользователя два числа и проверяет, делится ли первое число на второе без остатка. Программа должна выводить соответствующее сообщение о результате.
10. Определение сезона по месяцу:

Напишите программу, которая запрашивает у пользователя номер месяца (от 1 до 12) и выводит, к какому сезону этот месяц относится. Программа должна корректно обрабатывать неправильный ввод (например, если пользователь введет число за пределами диапазона).'''

url = "http://62.113.108.33/vebinar-v1/check_dz"
headers = {"Content-Type": "application/json",
    "Authorization": '89e768ed-4bc1-4e12-9139-45023e487b78'
    }
data = {
    "tema": tema,
    "dz": dz,
    "resheniye": resheniye
}

response = requests.post(url, headers=headers, data=json.dumps(data))
if response.status_code == 200:
  display(Markdown(json.loads(response.text)['choices'][0]['message']['content']))
  # print(json.loads(response.text)['choices'][0]['message']['content'])
else:
  print("Возникла ошибка:", response.status_code)
  print(response.text)

### Оценка решения студента

#### **Задачи уровня lite:**

**1. Простой калькулятор:**

```python
num1 = input("Введите первое число: ")
num2 = input("Введите второе число: ")
result = num1 + num2
print("Результат сложения:", result)
```

**Ошибки:**
- `input` возвращает строку, поэтому сложение строк будет некорректным для чисел. Необходимо преобразовать ввод в числа с помощью `int()` или `float()`.

**Рекомендации:**
- Преобразовать ввод в числа перед сложением.

**Исправленный код:**
```python
num1 = float(input("Введите первое число: "))
num2 = float(input("Введите второе число: "))
result = num1 + num2
print("Результат сложения:", result)
```

**2. Проверка четности числа:**

```python
number = input("Введите число: ")
if number % 2 == 0:
    print("Число четное")
else:
    print("Число нечетное")
```

**Ошибки:**
- `input` возвращает строку, поэтому операция `%` не будет работать. Необходимо преобразовать ввод в число.

**Рекомендации:**
- Преобразовать ввод в число перед проверкой.

**Исправленный код:**
```python
number = int(input("Введите число: "))
if number % 2 == 0:
    print("Число четное")
else:
    print("Число нечетное")
```

**3. Подсчет количества слов:**

```python
sentence = input("Введите предложение: ")
words = sentence.split()
print("Количество слов в предложении:", len(words))
```

**Ошибки:**
- Нет ошибок.

**Рекомендации:**
- Код корректен.

**4. Проверка на большее число:**

```python
num1 = input("Введите первое число: ")
num2 = input("Введите второе число: ")
if num1 > num2:
    print("Первое число больше")
elif num1 < num2:
    print("Второе число больше")
else:
    print("Числа равны")
```

**Ошибки:**
- `input` возвращает строку, поэтому сравнение строк будет некорректным для чисел. Необходимо преобразовать ввод в числа.

**Рекомендации:**
- Преобразовать ввод в числа перед сравнением.

**Исправленный код:**
```python
num1 = float(input("Введите первое число: "))
num2 = float(input("Введите второе число: "))
if num1 > num2:
    print("Первое число больше")
elif num1 < num2:
    print("Второе число больше")
else:
    print("Числа равны")
```

**5. Сумма чисел до N:**

```python
N = input("Введите число N: ")
sum = 0
for i in range(N):  # Ошибка: range() ожидает целое число, а не строку
    sum += i
print("Сумма чисел до", N, ":", sum)
```

**Ошибки:**
- `input` возвращает строку, поэтому `range()` не будет работать. Необходимо преобразовать ввод в число.
- Сумма чисел до N должна включать само число N.

**Рекомендации:**
- Преобразовать ввод в число перед использованием в `range()`.

**Исправленный код:**
```python
N = int(input("Введите число N: "))
sum = 0
for i in range(1, N + 1):
    sum += i
print("Сумма чисел до", N, ":", sum)
```

#### **Задачи уровня pro:**

**6. Калькулятор времени:**

```python
seconds = int(input("Введите количество секунд: "))
hours = seconds // 3600
minutes = (seconds - hours * 3600) // 60
seconds = seconds % 60
print(f"Время: {hours}:{minutes}:{seconds}")
```

**Ошибки:**
- Нет ошибок.

**Рекомендации:**
- Код корректен.

**7. Расчет стоимости покупки:**

```python
item_price = float(input("Введите стоимость товара: "))
quantity = int(input("Введите количество товаров: "))
total_cost = item_price * quantity
print("Общая стоимость покупки: ", total_cost)
```

**Ошибки:**
- Нет ошибок.

**Рекомендации:**
- Код корректен.

**8. Конвертер валют:**

```python
usd_amount = float(input("Введите сумму в долларах: "))
exchange_rate = 0.85
euro_amount = usd_amount * exchange_rate
print("Сумма в евро: ", euro_amount)
```

**Ошибки:**
- Нет ошибок.

**Рекомендации:**
- Код корректен.

**9. Проверка делимости чисел:**

```python
num1 = int(input("Введите первое число: "))
num2 = int(input("Введите второе число: "))
if num1 % num2 == 1:
    print(f"{num1} делится на {num2} без остатка")
else:
    print(f"{num1} не делится на {num2} без остатка")
```

**Ошибки:**
- Условие `if num1 % num2 == 1` неверно. Для проверки делимости без остатка нужно использовать `if num1 % num2 == 0`.

**Рекомендации:**
- Исправить условие проверки делимости.

**Исправленный код:**
```python
num1 = int(input("Введите первое число: "))
num2 = int(input("Введите второе число: "))
if num1 % num2 == 0:
    print(f"{num1} делится на {num2} без остатка")
else:
    print(f"{num1} не делится на {num2} без остатка")
```

**10. Определение сезона по месяцу:**

```python
month = int(input("Введите номер месяца (от 1 до 12): "))
if month == 12 or month <= 1:
    print("Зима")
elif month >= 3 and month <= 5:
    print("Весна")
elif month >= 6 and month <= 8:
    print("Лето")
elif month >= 9 and month <= 11:
    print("Осень")
else:
    print("Ошибка: введите число от 1 до 12")
```

**Ошибки:**
- Условие `if month == 12 or month <= 1` неверно. Январь и февраль также относятся к зиме.

**Рекомендации:**
- Исправить условие для зимы.

**Исправленный код:**
```python
month = int(input("Введите номер месяца (от 1 до 12): "))
if month == 12 or month <= 2:
    print("Зима")
elif month >= 3 and month <= 5:
    print("Весна")
elif month >= 6 and month <= 8:
    print("Лето")
elif month >= 9 and month <= 11:
    print("Осень")
else:
    print("Ошибка: введите число от 1 до 12")
```

### Заключение

Студент продемонстрировал базовые знания Python, но допустил несколько ошибок, связанных с преобразованием типов данных и логическими условиями. 

**Индивидуальные рекомендации:**
1. Обратить внимание на преобразование типов данных при вводе.
2. Тщательно проверять логические условия.
3. Запускать и тестировать код перед сдачей задания.

**Оценка: 6/10**

Студент справился с большинством задач, но допустил несколько критических ошибок, которые необходимо исправить.